In [7]:
import pandas as pd
import numpy as np
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.models import load_model
import urllib.request
import zipfile
import os
from keras.models import Sequential
from keras.layers import Embedding,Bidirectional,LSTM,GRU,Dense
import nltk
from nltk.tokenize import word_tokenize
import warnings
import tensorflow as tf
nltk.download('punkt')
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [25]:
df = pd.read_csv("combined.csv")

In [26]:
#!/usr/bin/env python
import re

# get rid of emojis
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)


In [29]:
# loop through posts, delete emojis, any random codes, punctuation
marks = '''!()-[]{};?@#$%:'"\,./^&*_\n'''

for _, row in df.iterrows():
  row["post"]= deEmojify(row["post"])
  row["post"] = row["post"].replace("&amp;#x200B;", "")
  
  for char in row["post"]:
    if char in marks:
      row["post"] = row["post"].replace(char, "")
  

In [37]:
pd.set_option('display.max_colwidth', 400)
df.head(8)

,subreddit,post
0,EDAnonymous,I ate slightly over maintenance tonight and even though I know that I purposefully did this because of my health I still want to die I feel like a fat fuck even though I have a BMI of ~21
1,EDAnonymous,How often do you guys have a bowel movement Ive been struggling with pretty intense onoff constipation for about 2+ months now but for the past three or so weeks Ive been extreeemeeelllyy constipated to the point where I can FEEL the poop in my body but just dont get the urge for it to come out So if I get scared enough I end up using laxatives around the 57 day mark of not having a movement ...
2,EDAnonymous,DAE get mad at their stomach I was on the bus home from work and my stomach grumbled and cramped apparently unsatisfied with my daily offering of shredded carrots and a protein bar Im 90 sure the guy in the aisle across from me heard me mutter fuck you youve had enough OopsBut that got me thinking of how often I target anger towards my stomachhunger cues for having the absolute AUDACITY to ask...
3,EDAnonymous,I fucked up and purged for the first time This is bad I know purging is bad for me Ive read countless horror stories of purging going wrong and all of the ways it takes a toll on your body I still found myself sticking my fingers down my throat for an hour trying to make myself sick I moved onto a foreign object I dont wanna name it but if you know you know With tears streaming down my face sn...
4,EDAnonymous,Boyfriend prefers underweight girls and now I want to die My boyfriend and I were talking about my eating disorder relapse which led to me discussing how I want to look super underweight Right after that was brought up he was was like “I’m not into super underweight girls I’m into fit girls” so I decide to show him pictures of girls I use as thinspo amp girls that are fit He immediately takes ...
5,EDAnonymous,Just Discovered this Sub As the title says I just discovered this sub an hour ago after googling binge eating therapy redditI dont think I have an ED but I know have bad tendencies and cravings I though that reading the posts here would give me some context from the perspective of those with real strugglesAnd then I read one word that I had never heard before but I instantly knew the definitio...
6,EDAnonymous,Old coworker told me I look “healthier” She worked with me when I was at a lw and tried to get me to eat more back then when she noticed Today she came in and we were talking and she asked me about and and told me I don’t look at skinny and that I look healthier than I was Lolololololololololihatemyself And food But this is great motivation to get back on track
7,EDAnonymous,i am thoroughly drunk and got the courage to post here so hi friends hello guys i’ve been lurking and i seriously love this community i used to post a bit in proed before the whole debacle and i just wsnted to share that my friends and i got drunk on franzia and i ate a sushi roll and as of right now i only care a little that i’m chunky and not as much as normal


In [55]:
x_train=[]
y_train=[]

for _, row in df.iterrows():
    y_train.append(row[0].strip())
    x_train.append(row[1])

In [57]:
import random
random.shuffle(x_train)
random.shuffle(y_train)

In [58]:
x_test = x_train[10000:]
y_test = y_train[10000:]
x_train = x_train[:10000]
y_train = y_train[:10000]

['adhd',
 'adhd',
 'autism',
 'autism',
 'depression',
 'bipolarreddit',
 'bipolarreddit',
 'schizophrenia',
 'EDAnonymous',
 'schizophrenia',
 'alcoholism',
 'schizophrenia',
 'adhd',
 'depression',
 'ptsd',
 'EDAnonymous',
 'EDAnonymous',
 'autism',
 'schizophrenia',
 'depression',
 'adhd',
 'ptsd',
 'EDAnonymous',
 'depression',
 'alcoholism',
 'ptsd',
 'autism',
 'adhd',
 'ptsd',
 'schizophrenia',
 'depression',
 'depression',
 'EDAnonymous',
 'bipolarreddit',
 'bipolarreddit',
 'bipolarreddit',
 'depression',
 'addiction',
 'ptsd',
 'bipolarreddit',
 'adhd',
 'EDAnonymous',
 'depression',
 'alcoholism',
 'adhd',
 'depression',
 'adhd',
 'adhd',
 'addiction',
 'bipolarreddit',
 'autism',
 'EDAnonymous',
 'depression',
 'autism',
 'adhd',
 'bipolarreddit',
 'addiction',
 'depression',
 'depression',
 'ptsd',
 'alcoholism',
 'schizophrenia',
 'adhd',
 'addiction',
 'alcoholism',
 'ptsd',
 'depression',
 'alcoholism',
 'adhd',
 'EDAnonymous',
 'depression',
 'addiction',
 'adhd',
 'al

In [62]:
data_train=pd.DataFrame({'Text':x_train,'Condition':y_train})
data_test=pd.DataFrame({'Text':x_test,'Condition':y_test})
data=data_train.append(data_test,ignore_index=True)

In [60]:
df["subreddit"].unique()

array(['EDAnonymous', 'subreddit', 'addiction', 'adhd', 'alcoholism',
       'autism', 'bipolarreddit', 'depression', 'ptsd', 'schizophrenia'],
      dtype=object)

In [63]:
def clean_text(data):
    data=re.sub(r"(#[\d\w\.]+)", '', data)
    data=re.sub(r"(@[\d\w\.]+)", '', data)
    data=word_tokenize(data)
    return data
texts=[' '.join(clean_text(text)) for text in data.Text]
texts_train=[' '.join(clean_text(text)) for text in x_train]
texts_test=[' '.join(clean_text(text)) for text in x_test]

In [64]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(texts)
sequence_train=tokenizer.texts_to_sequences(texts_train)
sequence_test=tokenizer.texts_to_sequences(texts_test)
index_of_words=tokenizer.word_index
vocab_size=len(index_of_words)+1

In [69]:
num_classes=10
embed_num_dims=300
max_seq_len=500
class_names=['EDAnonymous','subreddit','addiction','adhd','alcoholism','autism','bipolarreddit','depression','ptsd','schizophrenia']
X_train_pad=pad_sequences(sequence_train,maxlen=max_seq_len)
X_test_pad=pad_sequences(sequence_test,maxlen=max_seq_len)
encoding={'EDAnonymous':0,'subreddit':1,'addiction':2,'adhd':3,'alcoholism':4,'autism':5,'bipolarreddit':6,'depression':7,'ptsd':8,'schizophrenia':9}
y_train=[encoding[x] for x in data_train.Condition]
y_test=[encoding[x] for x in data_test.Condition]
y_train=to_categorical(y_train)
y_test=to_categorical(y_test)

In [66]:
def create_embedding_matrix(filepath,word_index,embedding_dim):
    vocab_size=len(word_index)+1
    embedding_matrix=np.zeros((vocab_size,embedding_dim))
    with open(filepath, encoding="utf8") as f:
        for line in f:
            word,*vector=line.split()
            if word in word_index:
                idx=word_index[word]
                embedding_matrix[idx] = np.array(vector,dtype=np.float32)[:embedding_dim]
    return embedding_matrix

fname='wiki-news-300d-1M.vec'
embedd_matrix=create_embedding_matrix(fname,index_of_words,embed_num_dims)

In [70]:
embedd_layer=Embedding(vocab_size,embed_num_dims,input_length=max_seq_len,weights=[embedd_matrix],trainable=False)
gru_output_size=128
bidirectional=True
model=Sequential()
model.add(embedd_layer)
model.add(Bidirectional(GRU(units=gru_output_size,dropout=0.2,recurrent_dropout=0.2)))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [71]:
batch_size=128
epochs=8
hist=model.fit(X_train_pad,y_train,batch_size=batch_size,epochs=epochs,validation_data=(X_test_pad,y_test))

Epoch 1/8
79/79 [==============================] - 232s 3s/step - loss: 2.1919 - accuracy: 0.1541 - val_loss: 2.1794 - val_accuracy: 0.1578
Epoch 2/8
79/79 [==============================] - 228s 3s/step - loss: 2.1636 - accuracy: 0.1616 - val_loss: 2.1779 - val_accuracy: 0.1581
Epoch 3/8
79/79 [==============================] - 227s 3s/step - loss: 2.1495 - accuracy: 0.1718 - val_loss: 2.1854 - val_accuracy: 0.1585
Epoch 4/8
47/79 [================>.............] - ETA: 1:29 - loss: 2.1385 - accuracy: 0.1840

KeyboardInterrupt: 

In [81]:
print("Where do I find support as a parent of an addict My son is an addict He was staying at a homeless shelter for a couple weeks but then they told him he couldn’t stay there last night I don’t understand the rules especially when it’s 20 outside He cannot stay with me but I felt extremely guilty")

Where do I find support as a parent of an addict My son is an addict He was staying at a homeless shelter for a couple weeks but then they told him he couldn’t stay there last night I don’t understand the rules especially when it’s 20 outside He cannot stay with me but I felt extremely guilty


In [87]:
message=['I used to be so thin and then I moved country and Im almost back to square 1 again  2017  January I weighed in at 65kg2017  August I weighed in at 55kg2018  January I weighed in at 60kg2018  September I weighed in at 59kg2019  February I dont weigh myself but I just took a nude for my boyfriend and Im gonna hazard a guess that Im 65kg Im so sad about it I am so so so mad at myself My goal was 52kg I was SO CLOSE Slow and steady ha Wins the race I guess Today I ran 12km and did weights Tomorrow Im going to do my 500kcal workout it probably doesnt burn that much but it certainly gets my heart rate going and its a lot of bodyweight exercises which helps Yesterday I consumed 512 kcal today 921kcalGod I hate myself so much']
seq=tokenizer.texts_to_sequences(message)
padded=pad_sequences(seq,maxlen=max_seq_len)
pred=model.predict(padded)
print('Message:'+str(message))
print('Emotion:',class_names[np.argmax(pred)])

Message:['I used to be so thin and then I moved country and Im almost back to square 1 again  2017  January I weighed in at 65kg2017  August I weighed in at 55kg2018  January I weighed in at 60kg2018  September I weighed in at 59kg2019  February I dont weigh myself but I just took a nude for my boyfriend and Im gonna hazard a guess that Im 65kg Im so sad about it I am so so so mad at myself My goal was 52kg I was SO CLOSE Slow and steady ha Wins the race I guess Today I ran 12km and did weights Tomorrow Im going to do my 500kcal workout it probably doesnt burn that much but it certainly gets my heart rate going and its a lot of bodyweight exercises which helps Yesterday I consumed 512 kcal today 921kcalGod I hate myself so much']
Emotion: adhd


In [43]:
x_train

['u', 'o']

In [80]:
x_train

['Neurotypical parents of autistic kids stop dismissing the things autistic people have to say My god the amount of autism mums on this subreddit is infuriating Im not talking about the ones who want advice that is completely fine Im talking about the ones who think they are more of an expert on autistic people than actual autistic people areIts usually the type of parent who uses their child as sympathy bait and wont shut up about wanting to cure them If you want to cure your child of autism you are a bad parent End of ',
 'Why does alcohol make me feel uneasy now Im 17 Now i already know what yall gonna say but hear me out For 2 years i was a pretty heavy drinker and i became very tolerant With some help i was able to quit weed was my alternative Ofcourse every now and then il still drink on a night out but Ive noticed that liquor cocktails and flavored drinks would make my stomach upset just after a few drinks or shotsBeer now makes me gassy and a big burp would build up but i cant 